# AWS_Autoclassification_Inbound_Archive_End_Month_QA

### Create Excel file

In [ ]:
import boto3
import pandas as pd

s3_client = boto3.client("s3")
bucket_name = "agropur-global-nonprod-account-staging-sapco"
prefix = "COPIA/Inbound/"
#prefix = "SAPCO/Inbound/"

response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
prefixes = [prefix["Prefix"] for prefix in response.get("CommonPrefixes")]

# Filtrar los archivos en cada subdirectorio
filtered_files = []
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    files = response.get("Contents")
    filtered_files.extend([file["Key"] for file in files if file["Key"].endswith(".json.gz") and "_StagingDataLake_202306" not in file["Key"]])

# Crear un DataFrame con los archivos filtrados
df = pd.DataFrame(filtered_files, columns=["File"])

# Guardar el DataFrame en un archivo Excel
excel_file = "Files_QA_to_Archive.xlsx"
df.to_excel(excel_file, index=False)

print(f"File saved {excel_file}")


## AWS_S3_PROD_AutoClassification_Archive_byYear_ByMonth

In [ ]:
import boto3
import pandas as pd
import datetime
import os

def classify_file(source_bucket, source_key, target_bucket):
    # Obtener el nombre del archivo y extraer el timestamp
    file_name = source_key.split('/')[-1]
    timestamp = file_name.split('_')[-1].split('.')[0]

    # Verificar si el archivo contiene el timestamp deseado
    if '202306' in timestamp:   # OJO Aqui deja de ejecutar el mes en curso
        print(f"File {source_key} do not be classified.")
        return

    # Convertir el timestamp en un objeto de fecha y obtener el año y mes correspondientes
    date = datetime.datetime.strptime(timestamp, "%Y%m%d%H%M%S")
    year = str(date.year)
    month = date.strftime('%m-%b')  #('%m-%B') Number-Month  2-Feb.

    # Construir la ruta de destino
    target_key = source_key.replace('COPIA/Inbound', f'COPIA/Archive/{year}/{month}')

    # Construir los comandos para mover el archivo
    src_path = f's3://{source_bucket}/{source_key}'
    tgt_path = f's3://{target_bucket}/{target_key}'
    cmd = f'aws s3 mv {src_path} {tgt_path}'

    # Mover el archivo al subdirectorio de destino
    status = os.system(cmd)

    if status == 0:
        print(f'File {source_key} classified at {target_key}.')
    else:
        print(f'Error al transferir el archivo {source_key}.')

def main():
    # Definir los nombres de los cubos y la ruta de origen
    source_bucket = 'agropur-global-nonprod-account-staging-sapco'
    source_prefix = 'COPIA/Inbound/'
    target_bucket = 'agropur-global-nonprod-account-staging-sapco'

    # Leer el archivo Excel con la lista de archivos filtrados
    excel_file = 'Files_QA_to_Archive.xlsx'
    df = pd.read_excel(excel_file)
    file_list = df['File'].tolist()

    # Contador de archivos transferidos
    transfer_count = 0

    for file in file_list:
        # Obtener la ruta de origen y clasificar el archivo
        source_key = file
        if classify_file(source_bucket, source_key, target_bucket):
            transfer_count += 1
    
    print(f'Total files transfered: {transfer_count}')
            
if __name__ == '__main__':
    main()

Los dos en un solo script

In [ ]:
import boto3
import pandas as pd
import datetime

def classify_file(source_bucket, source_key, target_bucket):
    # Obtener el nombre del archivo y extraer el timestamp
    file_name = source_key.split('/')[-1]
    timestamp = file_name.split('_')[-1].split('.')[0]

    # Verificar si el archivo contiene el timestamp deseado y si no contiene "_StagingDataLake_202306"
    if '202307' in timestamp and '_StagingDataLake_202307' not in file_name:
        print(f"INFO: File {source_key} do not need to be classified.")
        return

    # Convertir el timestamp en un objeto de fecha y obtener el año y mes correspondientes
    date = datetime.datetime.strptime(timestamp, "%Y%m%d%H%M%S")
    year = str(date.year)
    month = date.strftime('%m-%b')

    # Construir la ruta de destino
    target_key = source_key.replace('COPIA/Inbound', f'COPIA/Archive/{year}/{month}')
    #target_key = source_key.replace('SAPCO/Inbound', f'SAPCO/Archive/{year}/{month}')

    # Mover el archivo al subdirectorio de destino
    s3_client = boto3.client('s3')
    s3_client.copy_object(Bucket=target_bucket, Key=target_key, CopySource={'Bucket': source_bucket, 'Key': source_key})
    s3_client.delete_object(Bucket=source_bucket, Key=source_key)

    print(f'INFO: File {source_key} classified at {target_key}.')

def lambda_handler(event, context):
    # Definir los nombres de los cubos y la ruta de origen
    source_bucket = 'agropur-global-nonprod-account-staging-sapco'
    source_prefix = 'COPIA/Inbound/'
    #source_prefix = 'SAPCO/Inbound/'
    target_bucket = 'agropur-global-nonprod-account-staging-sapco'

    # Leer el archivo Excel con la lista de archivos filtrados
    excel_file = 'Files_QA_to_Archive.xlsx'
    df = pd.read_excel(excel_file)
    file_list = df['File'].tolist()

    # Contador de archivos transferidos
    transfer_count = 0

    for file in file_list:
        # Obtener la ruta de origen y clasificar el archivo
        source_key = file
        classify_file(source_bucket, source_key, target_bucket)
        transfer_count += 1

    print(f'Total files transferred: {transfer_count}')

    return {
        'statusCode': 200,
        'body': 'Script executed successfully.'
    }
#agropur-global-prod-account-prod-sapco


## AWS LAMBDA Function (Combined)

Fecha timestamp colocada de forma Manual

In [ ]:
import boto3
import datetime

def classify_file(source_bucket, source_key, target_bucket):
    # Obtener el nombre del archivo y extraer el timestamp
    file_name = source_key.split('/')[-1]
    timestamp = file_name.split('_')[-1].split('.')[0]

    # Verificar si el archivo contiene el timestamp deseado y si no contiene "_StagingDataLake_202306"
    if not timestamp or ('202306' in timestamp and '_StagingDataLake_202306' not in file_name):
        print(f"INFO: File {source_key} do not need to be classified.")
        return
    
    # Convertir el timestamp en un objeto de fecha y obtener el año y mes correspondientes
    date = datetime.datetime.strptime(timestamp, "%Y%m%d%H%M%S")
    year = str(date.year)
    month = date.strftime('%m-%b')

    # Construir la ruta de destino
    target_key = source_key.replace('COPIA/Inbound', f'COPIA/Archive/{year}/{month}')
    #target_key = source_key.replace('SAPCO/Inbound', f'SAPCO/Archive/{year}/{month}')

    # Mover el archivo al subdirectorio de destino
    s3_client = boto3.client('s3')
    s3_client.copy_object(Bucket=target_bucket, Key=target_key, CopySource={'Bucket': source_bucket, 'Key': source_key})
    s3_client.delete_object(Bucket=source_bucket, Key=source_key)

    print(f'INFO: File {source_key} classified at {target_key}.')

def lambda_handler(event, context):
    # Definir los nombres de los cubos y la ruta de origen
    source_bucket = 'agropur-global-nonprod-account-staging-sapco'
    source_prefix = 'COPIA/Inbound/'
    target_bucket = 'agropur-global-nonprod-account-staging-sapco'

    # Obtener la lista de archivos del bucket
    s3_client = boto3.client('s3')
    response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix, Delimiter='/')
    files = response.get('Contents', [])

    # Contador de archivos transferidos
    transfer_count = 0

    for file in files:
        source_key = file['Key']
        classify_file(source_bucket, source_key, target_bucket)
        transfer_count += 1

    print(f'Total files transferred: {transfer_count}')

    return {
        'statusCode': 200,
        'body': 'Script executed successfully.'
    }

Obtener automaticamente timestamp del sistema

In [ ]:
import boto3
import datetime

def classify_file(source_bucket, source_key, target_bucket, current_month):
    # Obtener el nombre del archivo y extraer el timestamp
    file_name = source_key.split('/')[-1]
    timestamp = file_name.split('_')[-1].split('.')[0]

    # Verificar si el archivo contiene el timestamp deseado y si no contiene "_StagingDataLake_202306"
    if not timestamp or (current_month in timestamp and f"_StagingDataLake_{current_month}" not in file_name):
        print(f"INFO: File {source_key} does not need to be classified.")
        return
    
    # Convertir el timestamp en un objeto de fecha y obtener el año y mes correspondientes
    date = datetime.datetime.strptime(timestamp, "%Y%m%d%H%M%S")
    year = str(date.year)
    month = date.strftime('%m-%b')

    # Construir la ruta de destino
    target_key = source_key.replace('COPIA/Inbound', f'COPIA/Archive/{year}/{month}')
    #target_key = source_key.replace('SAPCO/Inbound', f'SAPCO/Archive/{year}/{month}')

    # Mover el archivo al subdirectorio de destino
    s3_client = boto3.client('s3')
    s3_client.copy_object(Bucket=target_bucket, Key=target_key, CopySource={'Bucket': source_bucket, 'Key': source_key})
    s3_client.delete_object(Bucket=source_bucket, Key=source_key)

    print(f'INFO: File {source_key} classified at {target_key}.')

def lambda_handler(event, context):
    current_month = datetime.datetime.now().strftime("%Y%m")
#     file_name = event['file_name']
#     timestamp = event['timestamp']
    
    # Definir los nombres de los cubos y la ruta de origen
    source_bucket = 'agropur-global-nonprod-account-staging-sapco'
    source_prefix = 'COPIA/Inbound/'
    target_bucket = 'agropur-global-nonprod-account-staging-sapco'

    # Obtener la lista de archivos del bucket
    s3_client = boto3.client('s3')
    response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix, Delimiter='/')
    files = response.get('Contents', [])

    # Contador de archivos transferidos
    transfer_count = 0

    for file in files:
        source_key = file['Key']
        classify_file(source_bucket, source_key, target_bucket, current_month)
        transfer_count += 1

    print(f'Total files transferred: {transfer_count}')

    return {
        'statusCode': 200,
        'body': 'Script executed successfully.'
    }


In [ ]:
import boto3
import datetime

def classify_file(source_bucket, source_key, target_bucket, current_month):
    # Obtener el nombre del archivo y extraer el timestamp
    file_name = source_key.split('/')[-1]
    timestamp = file_name.split('_')[-1].split('.')[0]

    # Verificar si el archivo contiene el timestamp deseado y si no contiene "_StagingDataLake_202306"
    if not timestamp or (current_month in timestamp and f"_StagingDataLake_{current_month}" not in file_name):
        print(f"INFO: File {source_key} does not need to be classified.")
        return
    
    # Convertir el timestamp en un objeto de fecha y obtener el año y mes correspondientes
    date = datetime.datetime.strptime(timestamp, "%Y%m%d%H%M%S")
    year = str(date.year)
    month = date.strftime('%m-%b')

    # Construir la ruta de destino
    target_key = source_key.replace('COPIA/Inbound', f'COPIA/Archive/{year}/{month}')
    #target_key = source_key.replace('SAPCO/Inbound', f'SAPCO/Archive/{year}/{month}')

    # Mover el archivo al subdirectorio de destino
    s3_client = boto3.client('s3')
    s3_client.copy_object(Bucket=target_bucket, Key=target_key, CopySource={'Bucket': source_bucket, 'Key': source_key})
    s3_client.delete_object(Bucket=source_bucket, Key=source_key)

    print(f'INFO: File {source_key} classified at {target_key}.')

def lambda_handler(event, context):
    current_month = datetime.datetime.now().strftime("%Y%m")
    
    # Definir los nombres de los cubos y la ruta de origen
    source_bucket = 'agropur-global-nonprod-account-staging-sapco'
    source_prefix = 'COPIA/Inbound/'
    target_bucket = 'agropur-global-nonprod-account-staging-sapco'

    # Obtener la lista de archivos del bucket
    s3_client = boto3.client('s3')
    response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix, Delimiter='/')
    files = response.get('Contents', [])

    # Contador de archivos transferidos
    transfer_count = 0

    for file in files:
        source_key = file['Key']
        classify_file(source_bucket, source_key, target_bucket, current_month)
        transfer_count += 1

    print(f'Total files transferred: {transfer_count}')

    return {
        'statusCode': 200,
        'body': 'Script executed successfully.'
    }
